# Caso Wines – Análisis PCA y Clusterización

Notebook reproducible para el caso *Wines*: análisis de componentes principales, selección de dimensiones, clusterización y comparación con segmentos reales de vinos.

In [ ]:
# 0) Importar librerías necesarias
# Comentario: en este bloque cargamos todas las librerías que usaremos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

%matplotlib inline
sns.set(style="whitegrid", context="notebook")

## 1. Carga y preparación de los datos

In [ ]:
# 1.1) Cargar datasets de características químicas y segmentos reales
# Suponemos que el notebook está en la misma carpeta que wine-data.csv y wine-segments.csv

data = pd.read_csv("wine-data.csv")
segments = pd.read_csv("wine-segments.csv")

# Renombrar columna de segmentos para mayor claridad
segments.columns = ["Cultivar"]

# Unir en un solo DataFrame por índice
data_full = data.copy()
data_full["Cultivar"] = segments["Cultivar"].values

data_full.head()

In [ ]:
# 1.2) Revisión rápida de estructura y estadísticos básicos

print("Dimensiones del dataset:", data_full.shape)
display(data_full.describe().T)

data_full.info()

## 2. Análisis de Componentes Principales (PCA)

En esta sección estandarizaremos las variables químicas, aplicaremos PCA y analizaremos la varianza explicada por cada componente.

In [ ]:
# 2.1) Separar variables explicativas (X) y variable de segmento (y)

feature_cols = data.columns.tolist()  # todas las columnas de wine-data son numéricas
X = data_full[feature_cols].values
y = data_full["Cultivar"].values

X[:3], y[:3]

In [ ]:
# 2.2) Estandarizar las variables
# Comentario: PCA es sensible a la escala de las variables,
# por lo que primero normalizamos a media 0 y varianza 1.

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled[:3]

In [ ]:
# 2.3) Ajustar PCA sin fijar número de componentes para ver toda la varianza

pca_full = PCA()
pca_full.fit(X_scaled)

explained_var_ratio = pca_full.explained_variance_ratio_
cum_explained_var = np.cumsum(explained_var_ratio)

pca_var_df = pd.DataFrame({
    "Componente": np.arange(1, len(explained_var_ratio) + 1),
    "Varianza_Explicada": explained_var_ratio,
    "Varianza_Acumulada": cum_explained_var
})

pca_var_df

In [ ]:
# 2.4) Gráfico de la varianza explicada (scree plot) y varianza acumulada

plt.figure(figsize=(8, 5))
plt.plot(pca_var_df["Componente"], pca_var_df["Varianza_Explicada"], marker="o", label="Individual")
plt.plot(pca_var_df["Componente"], pca_var_df["Varianza_Acumulada"], marker="s", label="Acumulada")
plt.xlabel("Componente principal")
plt.ylabel("Porcentaje de varianza explicada")
plt.title("Varianza explicada por componentes principales")
plt.xticks(pca_var_df["Componente"])
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 3. Selección de número de dimensiones

En este paso decidimos cuántos componentes principales conservar en el nuevo espacio dimensional.

In [ ]:
# 3.1) Ver la varianza acumulada para los primeros componentes

pca_var_df.head(10)

In [ ]:
# 3.2) Seleccionar la cantidad de dimensiones del nuevo espacio
# Comentario:
# - Para visualización bidimensional nos interesa usar 2 componentes.
# - Estos suelen explicar una fracción importante de la varianza total.
#   (ver tabla anterior de varianza acumulada).

n_components_pca = 2
print(f"Número de componentes seleccionados para el nuevo espacio: {n_components_pca}")

## 4. Reducción al nuevo espacio dimensional y visualización

Reducimos los datos al espacio de los primeros componentes principales seleccionados y graficamos los resultados.

In [ ]:
# 4.1) Aplicar PCA con el número de componentes seleccionado

pca = PCA(n_components=n_components_pca)
X_pca = pca.fit_transform(X_scaled)

pca_cols = [f"PC{i+1}" for i in range(n_components_pca)]
df_pca = pd.DataFrame(X_pca, columns=pca_cols)
df_pca["Cultivar"] = y

df_pca.head()

In [ ]:
# 4.2) Gráfico en el nuevo espacio PCA coloreado por Cultivar real

plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    df_pca["PC1"],
    df_pca["PC2"],
    c=df_pca["Cultivar"],
    cmap="tab10",
    s=50,
    edgecolor="k"
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Vinos en espacio PCA (PC1 vs PC2) – coloreado por Cultivar real")
plt.legend(*scatter.legend_elements(), title="Cultivar")
plt.tight_layout()
plt.show()

> **Pregunta:** ¿Se aprecian clusters en el espacio de PC1 y PC2?

A simple vista debieran observarse grupos parcialmente separados que corresponden aproximadamente a los 3 cultivares.

## 5. Clusterización en el espacio PCA

En esta sección aplicaremos un algoritmo de clusterización (K-Means) sobre los datos proyectados en el espacio PCA y seleccionaremos la cantidad de clusters.

In [ ]:
# 5.1) Evaluar distintos valores de k con K-Means usando el espacio PCA
# Comentario: usaremos tanto la inercia (método del codo) como el
# coeficiente de silueta promedio para ayudar a elegir la cantidad de clusters.

range_n_clusters = range(2, 8)  # probamos desde 2 hasta 7 clusters
inertias = []
silhouettes = []

for k in range_n_clusters:
    kmeans_k = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_k = kmeans_k.fit_predict(df_pca[pca_cols])
    inertias.append(kmeans_k.inertia_)
    silhouettes.append(silhouette_score(df_pca[pca_cols], labels_k))

results_df = pd.DataFrame({
    "k": list(range_n_clusters),
    "inertia": inertias,
    "silhouette": silhouettes
})

results_df

In [ ]:
# 5.2) Graficar inercia (método del codo) y coeficiente de silueta

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(results_df["k"], results_df["inertia"], marker="o")
axes[0].set_title("Método del codo (inercia)")
axes[0].set_xlabel("Número de clusters (k)")
axes[0].set_ylabel("Inercia")
axes[0].grid(True)

axes[1].plot(results_df["k"], results_df["silhouette"], marker="o", color="orange")
axes[1].set_title("Coeficiente de silueta promedio")
axes[1].set_xlabel("Número de clusters (k)")
axes[1].set_ylabel("Silueta")
axes[1].grid(True)

plt.tight_layout()
plt.show()

for k, s in zip(results_df["k"], results_df["silhouette"]):
    print(f"k = {k}, silueta promedio = {s:.3f}")

In [ ]:
# 5.3) Seleccionar cantidad de clusters (k) basada en la silueta
# Comentario: elegimos el k que maximiza el coeficiente de silueta.

best_k = int(results_df.loc[results_df["silhouette"].idxmax(), "k"])
print(f"Número de clusters seleccionado (según silueta): {best_k}")

# Ajustar K-Means final con best_k clusters en el espacio PCA
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
clusters_pca = kmeans_final.fit_predict(df_pca[pca_cols])

# Guardar los labels de cluster
df_pca["Cluster"] = clusters_pca
data_full["Cluster"] = clusters_pca  # marcar dataset original con el número de cluster

data_full.head()

In [ ]:
# 5.4) Visualizar los clusters encontrados en el espacio PCA

plt.figure(figsize=(8, 6))
scatter = plt.scatter(
    df_pca["PC1"],
    df_pca["PC2"],
    c=df_pca["Cluster"],
    cmap="tab10",
    s=50,
    edgecolor="k"
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"Clusters de K-Means en espacio PCA (k = {best_k})")
plt.legend(*scatter.legend_elements(), title="Cluster")
plt.tight_layout()
plt.show()

## 6. Comparación con segmentos reales y conclusiones

In [ ]:
# 6.1) Comparar clusters encontrados vs. Cultivar real

ct = pd.crosstab(data_full["Cultivar"], data_full["Cluster"], margins=True)
ct

In [ ]:
# 6.2) (Opcional) Medir calidad de la clusterización con un índice externo

ari = adjusted_rand_score(data_full["Cultivar"], data_full["Cluster"])
print(f"Índice de Rand Ajustado (ARI) entre clusters y cultivares reales: {ari:.3f}")

### 6.3) Comentarios finales

- El PCA nos permitió reducir la dimensionalidad del problema y visualizar las observaciones en 2 componentes principales, manteniendo gran parte de la varianza original.
- Sobre este espacio reducido, K-Means encontró grupos basados en similitud de las características químicas de los vinos.
- La comparación con los cultivares reales (tabla de contingencia y ARI) nos da una idea de qué tan bien la clusterización no supervisada recupera la estructura "verdadera" de las clases.
- Un ARI cercano a 1 indica muy buen acuerdo; valores cercanos a 0 indican que los clusters no se parecen mucho a los cultivares reales.
- En un contexto real, estos análisis ayudan a entender si la segmentación basada solo en características químicas es suficiente para distinguir tipos de vino o si se requieren más variables o modelos más sofisticados.

### 6.4) Preguntas para resolver con este caso

1. Usando la tabla `pca_var_df`, ¿cuánta varianza explica el **primer componente (PC1)**? Escribe el valor en porcentaje.  
   *Tip:* Ejecuta la celda donde se muestra `pca_var_df` y toma el valor de la columna `Varianza_Explicada` para `Componente = 1` (primera fila). Luego multiplícalo por 100 para obtener el porcentaje.

2. Según `pca_var_df`, ¿cuánta **varianza acumulada** explican **PC1 y PC2 juntos**? ¿Es más del 50%?  
   *Tip:* En la misma tabla `pca_var_df`, mira la fila donde `Componente = 2` y toma el valor de la columna `Varianza_Acumulada`. Compáralo con 0.50 (50%).

3. Mira el gráfico PC1 vs PC2 coloreado por `Cultivar`: ¿se distinguen **aproximadamente 3 grupos** o los puntos se ven muy mezclados? Describe brevemente lo que observas.  
   *Tip:* Vuelve a ejecutar la celda del gráfico de PCA (PC1 vs PC2). Fíjate si los colores (cultivares) forman nubes separadas o si se superponen mucho.

4. Observa la tabla de resultados de `results_df`: ¿para qué valor de **k** el coeficiente de **silueta** es mayor? Escribe ese valor de k.  
   *Tip:* Ejecuta la celda que imprime `results_df` (o el bucle que muestra `k` y silueta). Busca la fila con el valor más alto en la columna `silhouette` y anota el `k` correspondiente.

5. Con la tabla de contingencia entre `Cultivar` y `Cluster`, responde: ¿para qué **Cultivar** parece haber un cluster donde la mayoría de sus vinos caen en la misma columna (cluster)?  
   *Tip:* Ejecuta la celda que calcula `ct = pd.crosstab(...)`. Para cada fila (Cultivar), identifica la columna (Cluster) con el número más alto. Esa combinación indica buena correspondencia.

6. Mira el valor del Índice de Rand Ajustado (ARI) que se imprime: si está **cerca de 1**, ¿qué significa? ¿Y si estuviera **cerca de 0**? Explica en una o dos frases.  
   *Tip:* Revisa la celda donde se imprime el `ari`. Piensa en “1” como *acuerdo perfecto* entre clusters y cultivares, y en “0” como *acuerdo similar al azar*.

**Interpretación del ARI**  
   - **ARI alto (cerca de 1)**: los clusters de K-Means se parecen mucho a los cultivares reales → **muy buena calidad** de clusterización.  
   - **ARI medio**: hay cierta relación entre clusters y cultivares, pero con bastantes errores → **calidad moderada**.  
   - **ARI bajo (cerca de 0)**: los clusters se parecen poco a los cultivares → la clusterización no está capturando bien la estructura real.